# Prerequisite Code

In [3]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

# Hyperparameters

In [4]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

# Optimization Loop

Each epoch consists of two main parts:
* The Train Loop - iterate over the training dataset and try to converge to optimal parameters.
* The Validation/Test Loop - iterate over the test dataset to check if model performance is improving.

## Loss Function

In [5]:
# Initialize the loss function
loss_fn = nn.CrossEntropyLoss()

## Optimizer

In [6]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# Full Implementation

In [7]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [9]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.259480  [    0/60000]
loss: 2.246093  [ 6400/60000]
loss: 2.215417  [12800/60000]
loss: 2.214186  [19200/60000]
loss: 2.192498  [25600/60000]
loss: 2.154488  [32000/60000]
loss: 2.170841  [38400/60000]
loss: 2.125826  [44800/60000]
loss: 2.128521  [51200/60000]
loss: 2.075349  [57600/60000]
Test Error: 
 Accuracy: 55.0%, Avg loss: 2.076799 

Epoch 2
-------------------------------
loss: 2.099828  [    0/60000]
loss: 2.080270  [ 6400/60000]
loss: 2.008087  [12800/60000]
loss: 2.027951  [19200/60000]
loss: 1.961297  [25600/60000]
loss: 1.900519  [32000/60000]
loss: 1.920021  [38400/60000]
loss: 1.831561  [44800/60000]
loss: 1.841998  [51200/60000]
loss: 1.744178  [57600/60000]
Test Error: 
 Accuracy: 61.5%, Avg loss: 1.755632 

Epoch 3
-------------------------------
loss: 1.805602  [    0/60000]
loss: 1.762864  [ 6400/60000]
loss: 1.638509  [12800/60000]
loss: 1.682565  [19200/60000]
loss: 1.569068  [25600/60000]
loss: 1.532861  [32000/600